In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale

# read the tsv
df0 = pd.read_csv('NSDUH_2015_Tab.tsv',sep='\t')
heroin_ever = df0['HEREVER']

# remove those never used heroin
indexes_to_remove = []
for i in range(len(heroin_ever)):
    if heroin_ever[i] != 1:
        indexes_to_remove.append(i)

df = df0.drop(index=indexes_to_remove)

/home/osari/dev/venvs/ipyth/lib64/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2514) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# did this to correct indexes
df.to_csv('her.tsv',sep='\t',index=False)
df = pd.read_csv('her.tsv',sep='\t')

In [8]:
# recency
heroin_recency = df['HERREC']
recency_score = []
for i in range(len(heroin_recency)):
    x = heroin_recency[i]
    if x == 1 or x == 11:
        rec_factor = 1
    elif x == 2 or x == 8:
        rec_factor = 0.5
    else:
        rec_factor = 0.2
    recency_score.append(rec_factor)

df['recency_score'] = recency_score

In [10]:
# frequency
heroin_frequency = df['HERYRTOT']
freq_score = []

for i in range(len(heroin_frequency)):
    if heroin_frequency[i] in range(1,366):
        freq_score.append(heroin_frequency[i])
    else:
        freq_score.append(0)

freq_score_mapped = np.interp(freq_score,(np.min(freq_score),np.max(freq_score)),(0.3,1))
df['freq_score_mapped'] = freq_score_mapped

In [12]:
# duration, 2015 is the year that the survey has been conducted. 
# IRHERYFU is the year of first use age.
# I did not include the month of first use as not many people responded.

duration = np.array(2015-df['IRHERYFU'])
duration_normalized = minmax_scale(duration)

duration_mapped = np.interp(duration_normalized,(np.min(duration_normalized),np.max(duration_normalized)),(0.3,1))
df['duration_score_mapped'] = duration_mapped

/home/osari/dev/venvs/ipyth/lib64/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [13]:
# RFD Score
rfd_score = []
        
for i in range(len(recency_score)):
    R = recency_score[i]
    F = freq_score_mapped[i]
    D = duration_mapped[i]
    RFD = R * ((F*4/5)+(D*3/5))
    if RFD < 1:
        rfd_score.append(RFD)
    else:
        rfd_score.append(1)

df['Final RFD Score'] = rfd_score

In [14]:
df.head()

,QUESTID2,FILEDATE,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,COUTYP4,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,recency_score,freq_score_mapped,duration_score_mapped,Final RFD Score
0,83427143,02/15/2018,1,99,99,12,9999,99,3,93,...,2,2,2,386.632560,40050,2,0.2,0.3,0.471698,0.104604
1,49280143,02/15/2018,1,99,99,13,9999,99,1,30,...,2,2,2,17039.787161,40029,1,0.2,0.3,0.762264,0.139472
2,74739243,02/15/2018,1,99,99,11,9999,99,1,30,...,3,2,2,555.029418,40044,2,0.2,0.3,0.366038,0.091925
3,55596343,02/15/2018,1,99,99,12,9999,99,1,30,...,1,2,2,1301.742010,40043,2,0.2,0.3,0.432075,0.099849
4,34869343,02/15/2018,1,99,99,10,9999,99,1,30,...,1,2,2,1147.347458,40035,2,0.2,0.3,0.352830,0.090340
